In [1]:
from transformers import pipeline,AutoModelForSeq2SeqLM,AutoTokenizer

In [13]:
model_ckpt = "suriya7/English-to-Tamil"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
pipelines =  pipeline("translation_en_to_ta",model=model,tokenizer=tokenizer)

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
text = "Hello,How are you ?"
print(pipelines(text))

[{'translation_text': 'ஹலோ! நீங்கள் எப்படி இருக்கிறீர்கள்?'}]


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00


# **Import neccessary libs**

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import torch

# **Id of model and dataset**

In [5]:
model_ckpt = "suriya7/English-to-Tamil"

In [6]:
dataset_id = "Hemanth-thunder/en_ta"

# **Load the dataset**

In [7]:

dataset = load_dataset(dataset_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

corpus.bcn.train 166k.csv:   0%|          | 0.00/99.2M [00:00<?, ?B/s]

en-ta train 118k.csv:   0%|          | 0.00/75.8M [00:00<?, ?B/s]

corpus.bcn.dev 1k.csv:   0%|          | 0.00/589k [00:00<?, ?B/s]

corpus.bcn.test 2k.csv:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/285630 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 285630
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'en', 'ta'],
        num_rows: 2000
    })
})

In [9]:

source_language = "en"
target_language = "ta"


In [ ]:
def preprocess(data):
  inputs = data[source_language]
  targets = data[target_language]

  model_inputs = tokenizer(inputs,max_length = 128,truncation=True)


  labels = tokenizer(targets,max_length=128,truncation=True)

  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [14]:
tokenized_datasets = dataset.map(preprocess,batched=True)

Map:   0%|          | 0/285630 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# **DataCollectorSeq2seq**

In [15]:
data_collector = DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='pt')

In [16]:
from transformers import TrainingArguments,Trainer

args = TrainingArguments(
    output_dir = "en-ta",
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    evaluation_strategy = "steps",
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets['test'],
    eval_dataset = tokenized_datasets['validation'],
    tokenizer = tokenizer,
    data_collator = data_collector
)

In [18]:
trainer.train()

Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


TrainOutput(global_step=125, training_loss=1.5964808349609374, metrics={'train_runtime': 312.3659, 'train_samples_per_second': 6.403, 'train_steps_per_second': 0.4, 'total_flos': 157159526227968.0, 'train_loss': 1.5964808349609374, 'epoch': 1.0})

In [19]:
model.save_pretrained("model_new")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [20]:
tokenizer.save_pretrained("tokenizer_new")

('tokenizer_new/tokenizer_config.json',
 'tokenizer_new/special_tokens_map.json',
 'tokenizer_new/vocab.json',
 'tokenizer_new/sentencepiece.bpe.model',
 'tokenizer_new/added_tokens.json')

In [22]:
pipelines =  pipeline("translation_en_to_ta",model="model_new",tokenizer='tokenizer_new')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [35]:
input = "Hello sam, I am going to my town."
res = pipelines(input)

In [36]:
res

[{'translation_text': 'ஹலோ சாம், நான் எனது நகருக்குச் செல்கிறேன்.'}]